### 数据预处理
1. 测试数据加载

In [59]:
import pandas as pd
import numpy as np
# 这里的编码类型可以用sublime打开查看左下角
base_test = pd.read_csv('tables/base-test.csv',engine='python',encoding='utf8')
knowledge_test = pd.read_csv('tables/knowledge-test.csv',engine='python',encoding='utf8')
money_test = pd.read_csv('tables/money-test.csv',engine='python',encoding='utf8')
year_test = pd.read_csv('tables/year-test.csv',engine='python',encoding='gb2312')

2. 处理base_test数据

    2.1 将中文特征编码

    2.2 去除无关信息

    2.3 缺失值处理(平均值)

In [60]:
map_industry = {'零售业':1,'服务业':2,'工业':3,'商业服务业':4,'社区服务':5,'交通运输业':6}
map_type_enterprise = {'有限责任公司':10,'合伙企业':20,'股份有限公司':30,'农民专业合作社':40,'集体所有制企业':50}
map_type_controller = {'自然人':10,'企业法人':20}
# 2.1 给中文编码
base_test['行业'] = base_test['行业'].map(map_industry)
base_test['企业类型'] = base_test['企业类型'].map(map_type_enterprise)
base_test['控制人类型'] = base_test['控制人类型'].map(map_type_controller)
# 2.2 去除无关信息
base_test_data = base_test.drop(columns=['区域'])
# 2.3 缺失值处理（平均值）
for column in list(base_test_data.columns[base_test_data.isnull().sum() > 0]):
    a = base_test_data[column].mean()
    base_test_data[column].fillna(a,inplace=True)

3. 处理knowledge_test数据

    3.1 缺失值处理(01数据类型，round四舍五入取值即可)
4. 处理money_test和year_test数据

    4.1 根据ID和year两个属性合并两个数据集

    4.2 提取15、16和17的数据

    4.3 将15、16和17年的数据合并成一张表

In [61]:
# 3.1 缺失值处理（平均数）
for column in list(knowledge_test.columns[knowledge_test.isnull().sum() > 0]):
    a = round(knowledge_test[column].mean())
    knowledge_test[column].fillna(a,inplace=True)
# 4.1 根据ID和year属性合并两个数据集
money_year_test = pd.merge(money_test,year_test,on=['ID','year'],how='outer')
# 4.2 提取15、16和17年的数据
money_year_test_15 = money_year_test.loc[money_year_test['year']==2015].add_suffix('_15')
money_year_test_15.rename(columns={'ID_15':'ID','year_15':'year'},inplace=True)
money_year_test_16 = money_year_test.loc[money_year_test['year']==2016].add_suffix('_16')
money_year_test_16.rename(columns={'ID_16':'ID','year_16':'year'},inplace=True)
money_year_test_17 = money_year_test.loc[money_year_test['year']==2017].add_suffix('_17')
money_year_test_17.rename(columns={'ID_17':'ID','year_17':'year'},inplace=True)
# 4.3 合并3年的数据
money_year_test_1516=pd.merge(money_year_test_15,money_year_test_16,on='ID',how='outer')
money_year_test_151617=pd.merge(money_year_test_1516,money_year_test_17,on='ID',how='outer')

5. 整合所有的表

    5.1 将base和knowledge表连接起来(采用内连接 x)

    5.2 将所有表连接

    5.3 去除多余的属性（year_x,year_y,year）

    5.4 填充缺失值（平均值）

In [62]:
# 5.1 将base和knowledge表连接起来(采用内连接)
base_knowledge_test = pd.merge(base_test_data,knowledge_test,on='ID',how='outer')
# 5.2 将所有表连接
test_data = pd.merge(money_year_test_151617,base_knowledge_test,on='ID',how='outer')
# 5.3 去除多余的属性（year_x,year_y,year）

In [63]:
test_data.info()
test_data = test_data.drop(columns=['year_x','year_y','year'])
# 5.4 填充缺失值（平均值）
for column in list(test_data.columns[test_data.isnull().sum() > 0]):
    a = int(test_data[column].mean())
    test_data[column].fillna(a,inplace=True)
# 最终训练数据
test_data.to_csv('tables/test.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7132 entries, 0 to 7131
Data columns (total 61 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              7132 non-null   int64  
 1   year_x          7131 non-null   float64
 2   债权融资额度_15       6977 non-null   float64
 3   债权融资成本_15       6986 non-null   float64
 4   股权融资额度_15       6967 non-null   float64
 5   股权融资成本_15       6973 non-null   float64
 6   内部融资和贸易融资额度_15  6973 non-null   float64
 7   内部融资和贸易融资成本_15  6979 non-null   float64
 8   项目融资和政策融资额度_15  6980 non-null   float64
 9   项目融资和政策融资成本_15  6967 non-null   float64
 10  从业人数_15         6983 non-null   float64
 11  资产总额_15         6978 non-null   float64
 12  负债总额_15         6991 non-null   float64
 13  营业总收入_15        6979 non-null   float64
 14  主营业务收入_15       6979 non-null   float64
 15  利润总额_15         7001 non-null   float64
 16  纳税总额_15         6997 non-null   float64
 17  所有者权益合计_15      6985 non-null   f